In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip3 install pydub

In [3]:
cd /content/drive/My\ Drive/tobigs/dnn_auto_encoder/music_makes_one

/content/drive/My Drive/tobigs/dnn_auto_encoder/music_makes_one


In [5]:
# from TFMusicAudioEncoder import process_data
import math
import tensorflow as tf
import numpy as np
from functools import partial
import importlib

#%%
### mp3로 변환하기(이미 했음)
# process_data.convert_mp3_to_wav()

#%%
LOSS_OUT_FILE = 'Epoch_Loss.txt'
# 파일 리스트 얻기
process_data.process_wav()


#%%
# Learning rate
lr = 0.0001

# L2 regularization
l2 = 0.0001

inputs = 12348
hidden_1_size = 8400
hidden_2_size = 3440
hidden_3_size = 2800

# Change the epochs variable to define the 
# number of times we iterate through all our batches
epochs = 10000

# Change the batch_size variable to define how many songs to load per batch
batch_size = 50

# Change the batches variable to change the number of batches you want per epoch
batches = 1

# Define our placeholder with shape [?, 12348]
X = tf.placeholder(tf.float32, shape=[None, inputs])
l2_regularizer = tf.contrib.layers.l2_regularizer(l2)

# partial 하면 저 변수가 들어간 상태로 둘 수 있지
autoencoder_dnn = partial(tf.layers.dense, 
						activation = tf.nn.elu,
						kernel_initializer = tf.contrib.layers.variance_scaling_initializer(),
						kernel_regularizer=  tf.contrib.layers.l2_regularizer(l2))

hidden_1 = autoencoder_dnn(X, hidden_1_size)
hidden_2 = autoencoder_dnn(hidden_1, hidden_2_size)
hidden_4 = autoencoder_dnn(hidden_2, hidden_3_size)
hidden_5 = autoencoder_dnn(hidden_4, hidden_2_size)
outputs =  autoencoder_dnn(hidden_5, inputs, activation=None)

reconstruction_loss = tf.reduce_mean(tf.square(outputs-X))

# 정규화에 더해지는 상수 값.
reg_loss = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.add_n([reconstruction_loss] + reg_loss)

# learning rate 두기
optimizer = tf.train.AdamOptimizer(lr)
training_op = optimizer.minimize(loss)
init = tf.global_variables_initializer()

#%%
def next_batch(c_batch, batch_size, sess):
	ch1_arr = []
	ch2_arr = []
	print(c_batch, batch_size)
	wav_arr_ch1, wav_arr_ch2, sample_rate = process_data.get_next_batch_original(c_batch, batch_size, sess)

	for sub_arr in wav_arr_ch1:
		batch_size_ch1 = math.floor(len(sub_arr)/inputs)
		sub_arr = sub_arr[:(batch_size_ch1*inputs)]
		ch1_arr.append(np.array(sub_arr).reshape(batch_size_ch1, inputs))

	for sub_arr in  wav_arr_ch2:
		batch_size_ch2 = math.floor(len(sub_arr)/inputs)
		sub_arr = sub_arr[:(batch_size_ch2*inputs)]
		ch2_arr.append(np.array(sub_arr).reshape(batch_size_ch2, inputs))

	return np.array(ch1_arr), np.array(ch2_arr), sample_rate

def next_batch2(c_batch, batch_size, sample_rate,sess):
	## 더 간단하게(file_to_data에서 전처리 했음)
	ch1_arr = []
	ch2_arr = []
	print(c_batch, batch_size)
	wav_arr_ch1, wav_arr_ch2, sample_rate = process_data.get_next_batch(c_batch, batch_size, sample_rate,sess)

	# inputs 단위로 잘라서 넣기
	for sub_arr in wav_arr_ch1:
		batch_size_ch1 = math.floor(len(sub_arr)/inputs)
		sub_arr = sub_arr[:(batch_size_ch1*inputs)]
		ch1_arr.append(np.array(sub_arr).reshape(batch_size_ch1, inputs))

	for sub_arr in  wav_arr_ch2:
		batch_size_ch2 = math.floor(len(sub_arr)/inputs)
		sub_arr = sub_arr[:(batch_size_ch2*inputs)]
		ch2_arr.append(np.array(sub_arr).reshape(batch_size_ch2, inputs))

	return np.array(ch1_arr), np.array(ch2_arr), sample_rate


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


# 처음부터 돌리기

In [0]:
cur_epoch = 0
##### Run training
with tf.Session() as sess:
	init.run()
	### saver 만들기
	saver = tf.train.Saver()
	sess.run(tf.global_variables_initializer())
	print('processing...')
	sample_rate = process_data.file_to_data(sess)
	# ckpt_path = saver.restore(sess, f"/content/drive/My Drive/tobigs/dnn_auto_encoder/saved/train{cur_epoch}")
	print('processing Done!')

	for epoch in range(cur_epoch+1,epochs):
		epoch_loss = []
		print("Epoch: " + str(epoch))
		for i in range(batches):
			ch1_song, ch2_song, sample_rate = next_batch2(i, batch_size, sample_rate,sess)
			# 두 채널 합치기
			total_songs = np.hstack([ch1_song, ch2_song])
			batch_loss = []

			for j in range(len(total_songs)):
				x_batch = total_songs[j]
				# 결과값과 비교하기
				_, l = sess.run([training_op, loss], feed_dict={X:x_batch})
				# 각 값들의 loss 구하기
				batch_loss.append(l)
				# print("Song loss: " + str(l))

			print("Curr Epoch: " + str(epoch) + " Curr Batch: " + str(i) + "/"+ str(batches))
			print("Batch Loss: " + str(np.mean(batch_loss)))
			epoch_loss.append(np.mean(batch_loss))
			
		print("Epoch Avg Loss: " + str(np.mean(epoch_loss)))

		if epoch % 500 == 0:
			ckpt_path = saver.save(sess, f'/content/drive/My Drive/tobigs/dnn_auto_encoder/saved/train{epoch}')

	ckpt_path = saver.save(sess, "/content/drive/My Drive/tobigs/dnn_auto_encoder/saved/train")
#9:10~9:30 100 epoch 간다.

processing...

Returning File: /content/drive/My Drive/tobigs/dnn_auto_encoder/music_makes_one/0.wav
Returning File: /content/drive/My Drive/tobigs/dnn_auto_encoder/music_makes_one/1.wav
Returning File: /content/drive/My Drive/tobigs/dnn_auto_encoder/music_makes_one/2.wav
Returning File: /content/drive/My Drive/tobigs/dnn_auto_encoder/music_makes_one/3.wav
Returning File: /content/drive/My Drive/tobigs/dnn_auto_encoder/music_makes_one/4.wav
Returning File: /content/drive/My Drive/tobigs/dnn_auto_encoder/music_makes_one/5.wav
Returning File: /content/drive/My Drive/tobigs/dnn_auto_encoder/music_makes_one/6.wav
Returning File: /content/drive/My Drive/tobigs/dnn_auto_encoder/music_makes_one/7.wav
Returning File: /content/drive/My Drive/tobigs/dnn_auto_encoder/music_makes_one/8.wav
Returning File: /content/drive/My Drive/tobigs/dnn_auto_encoder/music_makes_one/9.wav
Returning File: /content/drive/My Drive/tobigs/dnn_auto_encoder/music_makes_one/10.wav
Returning File: /content/drive/My Driv

# 중간 체크포인트부터

In [0]:
cur_epoch = 5000
##### Run training
with tf.Session() as sess:
	init.run()
	### saver 만들기
	saver = tf.train.Saver()
	sess.run(tf.global_variables_initializer())
	print('processing...')
	sample_rate = process_data.file_to_data(sess)
	ckpt_path = saver.restore(sess, f"/content/drive/My Drive/tobigs/dnn_auto_encoder/saved/train{cur_epoch}")
	print('processing Done!')

	for epoch in range(cur_epoch+1,epochs):
		epoch_loss = []
		print("Epoch: " + str(epoch))
		for i in range(batches):
			ch1_song, ch2_song, sample_rate = next_batch2(i, batch_size, sample_rate,sess)
			total_songs = np.hstack([ch1_song, ch2_song])
			batch_loss = []

			for j in range(len(total_songs)):
				x_batch = total_songs[j]
				_, l = sess.run([training_op, loss], feed_dict={X:x_batch})
				batch_loss.append(l)
				# print("Song loss: " + str(l))

			print("Curr Epoch: " + str(epoch) + " Curr Batch: " + str(i) + "/"+ str(batches))
			print("Batch Loss: " + str(np.mean(batch_loss)))
			epoch_loss.append(np.mean(batch_loss))
			
		print("Epoch Avg Loss: " + str(np.mean(epoch_loss)))

		if epoch % 200 == 0:
			ckpt_path = saver.save(sess, f'/content/drive/My Drive/tobigs/dnn_auto_encoder/saved/train{epoch}')

	ckpt_path = saver.save(sess, "/content/drive/My Drive/tobigs/dnn_auto_encoder/saved/train")
#9:10~9:30 100 epoch 간다.

# TEST 결과 확인

In [14]:
print(len(process_data.file_arr))
print(process_data.file_arr[5])

52
/content/drive/My Drive/tobigs/dnn_auto_encoder/music_makes_one/5.wav


In [17]:
##### test train 없는데이터로(200 epoch)
sample_rate = 44100
epoch = 1500
data_num = 51
from scipy.fftpack import rfft, irfft
from tensorflow.contrib import ffmpeg
import json 

with tf.Session() as sess:
	init.run()
	### saver 만들기
	saver = tf.train.Saver()
	sess.run(tf.global_variables_initializer())
	
	# sample_rate = process_data.file_to_data(sess)
	ckpt_path = saver.restore(sess, f"/content/drive/My Drive/tobigs/dnn_auto_encoder/saved/train{epoch}")

	# 학습하지 않은 데이터 받기
	ch1_song_new, ch2_song_new, sample_rate_new = next_batch(data_num, 1, sess)
	x_batch = np.hstack([ch1_song_new, ch2_song_new])[0]
	print("Sample rate: " + str(sample_rate_new)) 

	orig_song = []
	full_song = []
	evaluation = outputs.eval(feed_dict={X: x_batch})
	print("Output: " + str(evaluation)) 
	full_song.append(evaluation)
	orig_song.append(x_batch)
	print(len(full_song), len(full_song[0]))
	# Merge the nested arrays
	# 1*856 짜리인걸 한 array로 바꾼다!
	orig_song = np.hstack(orig_song)
	full_song = np.hstack(full_song)
	
	print('shape')
	print(orig_song.shape, full_song.shape)

	# Compute and split the channels
	orig_song_ch1 = orig_song[:math.floor(len(orig_song)/2)]
	orig_song_ch2 = orig_song[math.floor(len(orig_song)/2):]
	full_song_ch1 = full_song[:math.floor(len(full_song)/2)]
	full_song_ch2 = full_song[math.floor(len(full_song)/2):]
	print('next')
	print(np.hstack(full_song_ch1).shape, np.hstack(orig_song_ch1).shape)
	# print(np.hstack(np.hstack(full_song_ch1)).shape, full_song_ch1.shape)
	
	# Save both the untouched song and reconstructed song to the 'output' folder
	# process_data.save_to_wav(full_song_ch1, full_song_ch2, sample_rate, orig_song_ch1, orig_song_ch2, epoch, 'output', sess)


	# audio_arr_ch1 = irfft(np.hstack(np.hstack(full_song_ch1)))
	# audio_arr_ch2 = irfft(np.hstack(np.hstack(full_song_ch2)))

	# print('reconstruced done')
	# original_song_ch1 = irfft(np.hstack(np.hstack(orig_song_ch1)))
	# original_song_ch2 = irfft(np.hstack(np.hstack(orig_song_ch2)))

	audio_arr_ch1 = irfft(np.hstack(full_song_ch1))
	audio_arr_ch2 = irfft(np.hstack(full_song_ch2))

	print('reconstruced done')
	# original_song_ch1 = irfft(np.hstack(orig_song_ch1))
	# original_song_ch2 = irfft(np.hstack(orig_song_ch2))


	print('irfft done')
	# original_song = np.hstack(np.array((original_song_ch1, original_song_ch2)).T)
	audio_arr = np.hstack(np.array((audio_arr_ch1, audio_arr_ch2)).T)

	# print(original_song)
	w = np.linspace(0, sample_rate, len(audio_arr))
	w = w[0:len(audio_arr)]


	cols = 2
	rows = math.floor(len(audio_arr)/2)
	audio_arr = audio_arr.reshape(rows, cols)
	# original_song = original_song.reshape(rows, cols)

	wav_encoder = ffmpeg.encode_audio(
		audio_arr, file_format='wav', samples_per_second=sample_rate)
	# wav_encoder_orig = ffmpeg.encode_audio(
	# 	original_song, file_format='wav', samples_per_second=sample_rate)

	wav_file = sess.run(wav_encoder)
	# wav_orig = sess.run(wav_encoder_orig)
	open(f'/content/drive/My Drive/tobigs/dnn_auto_encoder/output/out_{data_num}_{epoch}.wav', 'wb').write(wav_file)
	# open('output/wav_orig.wav', 'wb').write(wav_orig)

# %%


INFO:tensorflow:Restoring parameters from /content/drive/My Drive/tobigs/dnn_auto_encoder/saved/train1500
51 1
Returning File: /content/drive/My Drive/tobigs/dnn_auto_encoder/music_makes_one/99.wav
Sample rate: 44100
Output: [[-7.72812012e+02 -1.94946014e+02 -3.57272552e+02 ...  1.61969681e+02
   1.49675430e+02 -3.50217957e+02]
 [ 8.55272034e+02 -1.45860925e+03 -8.96970947e+02 ...  1.15310959e+02
  -9.91639343e+02 -2.85739960e+02]
 [-2.74825409e+02 -1.36053406e+02  1.18918304e+02 ... -7.69390488e+01
   7.71388092e+01  2.08782700e+02]
 ...
 [ 2.98753202e-01 -7.69713223e-01  4.42849487e-01 ...  1.98937714e-01
  -9.37646747e-01  2.60397077e-01]
 [ 2.70886093e-01 -7.29571640e-01  4.55957651e-01 ...  1.96081042e-01
  -9.25953031e-01  2.50121117e-01]
 [ 3.47148836e-01 -7.29793251e-01  4.79579270e-01 ...  1.74841568e-01
  -9.30139661e-01  2.23104119e-01]]
1 856
shape
(856, 12348) (856, 12348)
next
(5284944,) (5284944,)
reconstruced done
irfft done


# test train에 있는 데이터로

In [10]:

##### test train 있는 데이터로(200 epoch)
sample_rate = 44100
epoch = 5000
data_num = 3
from scipy.fftpack import rfft, irfft
from tensorflow.contrib import ffmpeg
import json 

with tf.Session() as sess:
	init.run()
	### saver 만들기
	saver = tf.train.Saver()
	sess.run(tf.global_variables_initializer())
	
	# sample_rate = process_data.file_to_data(sess)
	ckpt_path = saver.restore(sess, f"/content/drive/My Drive/tobigs/dnn_auto_encoder/saved/train{epoch}")

	# 학습 된 곡   받기
	ch1_song_new, ch2_song_new, sample_rate_new = next_batch(2, 1, sess)
	x_batch = np.hstack([ch1_song_new, ch2_song_new])[0]
	print("Sample rate: " + str(sample_rate_new)) 

	orig_song = []
	full_song = []
	evaluation = outputs.eval(feed_dict={X: x_batch})
	print("Output: " + str(evaluation)) 
	full_song.append(evaluation)
	orig_song.append(x_batch)
	print(len(full_song), len(full_song[0]))
	# Merge the nested arrays
	# 1*856 짜리인걸 한 array로 바꾼다!
	orig_song = np.hstack(orig_song)
	full_song = np.hstack(full_song)
	
	print('shape')
	print(orig_song.shape, full_song.shape)

	# Compute and split the channels
	orig_song_ch1 = orig_song[:math.floor(len(orig_song)/2)]
	orig_song_ch2 = orig_song[math.floor(len(orig_song)/2):]
	full_song_ch1 = full_song[:math.floor(len(full_song)/2)]
	full_song_ch2 = full_song[math.floor(len(full_song)/2):]
	print('next')
	print(np.hstack(full_song_ch1).shape, np.hstack(orig_song_ch1).shape)
	# print(np.hstack(np.hstack(full_song_ch1)).shape, full_song_ch1.shape)
	
	# Save both the untouched song and reconstructed song to the 'output' folder
	# process_data.save_to_wav(full_song_ch1, full_song_ch2, sample_rate, orig_song_ch1, orig_song_ch2, epoch, 'output', sess)


	# audio_arr_ch1 = irfft(np.hstack(np.hstack(full_song_ch1)))
	# audio_arr_ch2 = irfft(np.hstack(np.hstack(full_song_ch2)))

	# print('reconstruced done')
	# original_song_ch1 = irfft(np.hstack(np.hstack(orig_song_ch1)))
	# original_song_ch2 = irfft(np.hstack(np.hstack(orig_song_ch2)))

	audio_arr_ch1 = irfft(np.hstack(full_song_ch1))
	audio_arr_ch2 = irfft(np.hstack(full_song_ch2))

	print('reconstruced done')
	# original_song_ch1 = irfft(np.hstack(orig_song_ch1))
	# original_song_ch2 = irfft(np.hstack(orig_song_ch2))


	print('irfft done')
	# original_song = np.hstack(np.array((original_song_ch1, original_song_ch2)).T)
	audio_arr = np.hstack(np.array((audio_arr_ch1, audio_arr_ch2)).T)

	# print(original_song)
	w = np.linspace(0, sample_rate, len(audio_arr))
	w = w[0:len(audio_arr)]


	cols = 2
	rows = math.floor(len(audio_arr)/2)
	audio_arr = audio_arr.reshape(rows, cols)
	# original_song = original_song.reshape(rows, cols)

	wav_encoder = ffmpeg.encode_audio(
		audio_arr, file_format='wav', samples_per_second=sample_rate)
	# wav_encoder_orig = ffmpeg.encode_audio(
	# 	original_song, file_format='wav', samples_per_second=sample_rate)

	wav_file = sess.run(wav_encoder)
	# wav_orig = sess.run(wav_encoder_orig)
	open(f'/content/drive/My Drive/tobigs/dnn_auto_encoder/output/out_{data_num}_{epoch}.wav', 'wb').write(wav_file)
	# open('output/wav_orig.wav', 'wb').write(wav_orig)

# %%


ValueError: ignored